In [ ]:
import pandas
import matplotlib
import requests
import warnings

import sys
sys.path.append('..')

from helper import load_data , load_repo , GIT_API
warnings.filterwarnings('ignore')

%matplotlib inline

import os
import json
import time

In [ ]:
json_store_dir = '..'+os.sep+'data'+os.sep+'json'

In [ ]:
testrepo = 'https://github.com/rust-lang/cargo'

ISSUE = 'issue'
PULL = 'pull'

time_sleep = 0.1

OPEN = 'open'
CLOSED = 'closed'

In [ ]:
# this function extracts number of open and closed issues of passed repository
def get_open_closed_issues(repo):
    add = repo + '/issues'
    time.sleep(time_sleep)
    req = requests.get(add)
    data = req.text
    
    open_issues = int(data.split('<svg class="octicon octicon-issue-opened"')[2].split('Open')[0].split('</svg>')[1].strip().replace(",",""))
    closed_issues = int(data.split('<svg class="octicon octicon-check"')[1].split('Closed')[0].split('</svg>')[1].strip().replace(",",""))
    
    return open_issues, closed_issues

In [ ]:
#this function fetches comment of issues and returns a dataframe of comments
def get_comments(repo,issue_id,is_pull_issue):
    df = pandas.DataFrame(columns=["project_name","issue_number","comment_id","user_login","created_at","updated_at","author_assoc"])
    
    repo_parts = repo.split('/')
    repo_full = repo_parts[-2]+'/'+repo_parts[-1]
    print(repo_full)
    
    if is_pull_issue==ISSUE:
        par_folder = json_store_dir+os.sep+'issue_comments'+os.sep+repo_full.replace('/',os.sep)
        json_store_location = par_folder+os.sep+str(issue_id)+'.json'
    elif is_pull_issue==PULL:
        par_folder = json_store_dir+os.sep+'pull_comments'+os.sep+repo_full.replace('/',os.sep)
        json_store_location = par_folder+os.sep+str(issue_id)+'.json'
        
    print(json_store_location)
    if os.path.exists(json_store_location):
        with open(json_store_location) as f:
            comments = json.load(f)
    else:
        issue_comment = 'https://api.github.com/repos/{}/{}/issues/{}/comments?client_id={}&client_secret={}'.format(
                repo.split('/')[3] ,
                repo.split('/')[4] ,
                issue_id,
                GIT_API[11].split(':')[0] , GIT_API[11].split(':')[1])
        print(issue_comment)
        time.sleep(time_sleep)
        req = requests.get(issue_comment)
        comments = req.json()
        if not os.path.exists(par_folder):
            os.makedirs(par_folder)
        with open(json_store_location, 'w') as outfile:
            json.dump(comments, outfile)
            
    for comment in comments:
        df = df.append({
            "project_name": repo,
            "issue_number": issue_id,
            "comment_id": comment['id'],
            "user_login": comment['user']['login'],
            "created_at": comment['created_at'],
            "updated_at": comment['updated_at'],
            "author_assoc": comment['author_association'] },ignore_index=True)
    return df

In [ ]:
# this function extracts number of open and closed pull requests of passed repository
def get_open_closed_pulls(repo):
    add = repo + '/pulls'
    time.sleep(time_sleep)
    req = requests.get(add)
    data = req.text
    
    open_issues = int(data.split('<svg class="octicon octicon-git-pull-request"')[2].split('Open')[0].split('</svg>')[1].strip().replace(",",""))
    closed_issues = int(data.split('<svg class="octicon octicon-check"')[1].split('Closed')[0].split('</svg>')[1].strip().replace(",",""))
    
    return open_issues, closed_issues

In [ ]:
#this function fetches review comment of pull request and returns a dataframe of review comments
def get_pull_comments(repo,pr_id):
    df = pandas.DataFrame(columns=["project_name",
                                   "pr_id",
                                   "pr_review_id",
                                   "comment_id",
                                   "commit_id",
                                   "original_commit_id",
                                   "user_login",
                                   "created_at",
                                   "updated_at",
                                   "author_assoc"])
    
    repo_parts = repo.split('/')
    repo_full = repo_parts[-2]+'/'+repo_parts[-1]
    print(repo_full)
    
    par_folder = json_store_dir+os.sep+'code_review_comments'+os.sep+repo_full.replace('/',os.sep)
    json_store_location = par_folder+os.sep+str(pr_id)+'.json'
        
    if os.path.exists(json_store_location):
        with open(json_store_location) as f:
            comments = json.load(f)
    else:
        pr_comment = 'https://api.github.com/repos/{}/{}/pulls/{}/comments?client_id={}&client_secret={}'.format(
                repo.split('/')[3] ,
                repo.split('/')[4] ,
                pr_id,
                GIT_API[11].split(':')[0] , GIT_API[11].split(':')[1])
        time.sleep(time_sleep)
        req = requests.get(pr_comment)
        comments = req.json()
        
        if not os.path.exists(par_folder):
            os.makedirs(par_folder)
        with open(json_store_location, 'w') as outfile:
            json.dump(comments, outfile)
        
        
    for comment in comments:
        df = df.append({
            "project_name": repo,
            "pr_id": pr_id,
            "pr_review_id" : comment['pull_request_review_id'],
            "comment_id": comment['id'],
            "commit_id" : comment['commit_id'],
            "original_commit_id" : comment['original_commit_id'],
            "user_login": comment['user']['login'],
            "created_at": comment['created_at'],
            "updated_at": comment['updated_at'],
            "author_assoc": comment['author_association'] },ignore_index=True)
    return df

# Dataframes

In [ ]:
issue_comments = pandas.DataFrame(columns=["project_name","issue_number","comment_id","user_login","created_at","updated_at","author_assoc"])
pulls_comments = pandas.DataFrame(columns=["project_name","issue_number","comment_id","user_login","created_at","updated_at","author_assoc"])
pulls_review_comments = pandas.DataFrame(columns=["project_name","pr_id","pr_review_id","comment_id","commit_id","original_commit_id","user_login","created_at","updated_at","author_assoc"])


In [ ]:
# extracting number of issues and prs for proposed repo
open_issue , closed_issue = get_open_closed_issues(testrepo)  
open_pr , closed_pr = get_open_closed_pulls(testrepo)  

# Open/Closed Issues Comments

In [ ]:
def get_issues_comments(repo,is_open_closed,issues_number):
    global issue_comments
    for i in range(1,(issues_number//25)+2):

        issues_page_url = repo + ('/issues?page={}&q=is%3Aissue+is%3A'+is_open_closed).format(i)
        print(issues_page_url)
        time.sleep(time_sleep)
        req = requests.get(issues_page_url)
        data = req.text
        issues = data.split('id="issue_')
        for issue in issues:
            if 'DOCTYPE' in issue:
                continue

            if 'octicon octicon-comment' in issue:
                issue_id = issue.split('"')[0]
                issue_comments = issue_comments.append(get_comments(repo,issue_id,ISSUE),ignore_index=True)
 
get_issues_comments(testrepo,OPEN,open_issue)
get_issues_comments(testrepo,CLOSED,closed_issue)

# Open/Closed Pull Request Comments

In [ ]:
def get_pulls_comments(repo,is_open_closed,pulls_number):
    pulls_comments = pandas.DataFrame(columns=["project_name","issue_number","comment_id","user_login","created_at","updated_at","author_assoc"])
    pulls_review_comments = pandas.DataFrame(columns=["project_name","pr_id","pr_review_id","comment_id","commit_id","original_commit_id","user_login","created_at","updated_at","author_assoc"])
    for i in range(1,(pulls_number//25)+2):
        pull_page_url = repo + ('/pulls?page={}&q=is%3Apr+is%3A'+is_open_closed).format(i)
        time.sleep(time_sleep)
        req = requests.get(pull_page_url)
        data = req.text
        issues = data.split('id="issue_')
        for issue in issues:
            if 'DOCTYPE' in issue:
                continue
            if 'octicon octicon-comment' in issue:
                pr_id = issue.split('"')[0]
                pulls_comments = pulls_comments.append(get_comments(repo,pr_id,PULL),ignore_index=True)
                pulls_review_comments = pulls_review_comments.append(get_pull_comments(repo,pr_id),ignore_index=True)
    return pulls_comments, pulls_review_comments
                
get_pulls_comments(testrepo,OPEN,open_pr)
get_pulls_comments(testrepo,CLOSED,closed_pr)